# Parcoursup dataset conversion to sentences and embeddings

### Import dataset : https://www.data.gouv.fr/fr/datasets/cartographie-des-formations-parcoursup/

In [55]:
import pandas as pd

In [56]:
formations_dataset = pd.read_csv('fr-esr-cartographie_formations_parcoursup.csv', delimiter=';')
formations_dataset = formations_dataset.drop(columns=['etab_uai', 'dataviz', 'gta',
                                                      'gti', 'etablissement_id_paysage',
                                                      'composante_id_paysage', 'code_formation',
                                                      'fl', 'nmc', 'rnd', 'tf'])

/tmp/ipykernel_36536/1273215455.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  formations_dataset = pd.read_csv('fr-esr-cartographie_formations_parcoursup.csv', delimiter=';')


In [57]:
def normalize_formations(df : pd.DataFrame):
    # int
    df['int'] = df['int'].fillna(0)
    df['int'] = df['int'].astype(str)
    df['int_f'] = 0
    df['int_m'] = 0
    df['int_f'] = df['int'].str.contains('filles').astype(int)
    df['int_m'] = df['int'].str.contains('garçons').astype(int)
    df = df.drop(columns=['int'])

    # app
    df['app'] = df['app'].fillna(0)
    df['app'] = df['app'].replace('Formations en apprentissage', 1)

    # tc
    df['tc'] = df['tc'].apply(lambda x: 1 if 'Privé' in x else 0)

    # aut
    df['aut'] = df['aut'].fillna(-1)
    df['aut'] = df['aut'].astype(str)
    df['aut'] = df['aut'].apply(lambda x: 1 if 'hors Parcoursup' in x else 0)
    df['aut'] = df['aut'].astype(int)

    # amg
    df_amg = pd.DataFrame(columns=['amg_p', 'amg_ed', 'amg_pd', 'amg_s', 'amg_a'])
    df_amg['amg_p'] = 0
    df_amg['amg_ed'] = 0
    df_amg['amg_pd'] = 0
    df_amg['amg_s'] = 0
    df_amg['amg_a'] = 0

    df_amg['amg_p'] = df['amg'].apply(lambda x: 1 if 'présentiel' in x else 0)
    df_amg['amg_ed'] = df['amg'].apply(lambda x: 1 if 'entièrement à distance' in x else 0)
    df_amg['amg_pd'] = df['amg'].apply(lambda x: 1 if 'partiellement à distance' in x else 0)
    df_amg['amg_s'] = df['amg'].apply(lambda x: 1 if 'sportifs' in x else 0)
    df_amg['amg_a'] = df['amg'].apply(lambda x: 1 if 'artistes' in x else 0)

    df = pd.concat([df, df_amg], axis=1)
    df = df.drop(columns=['amg'])

    return df

### Conversion to sentences for sentence comparison

In [64]:
def convert_to_description(row):
    # check nan
    if not type(row['commune']) == float:
        if row['commune'] not in row['etab_nom']:
            sent= f"{row['etab_nom']}, située en {row['departement']} à {row['commune']}, est une école qui propose la formation : {row['nm']}."
        else:
            sent= f"{row['etab_nom']}, située en {row['departement']}, est une école qui propose la formation : {row['nm']}."
    else:
        sent= f"{row['etab_nom']}, située en {row['departement']}, est une école qui propose la formation : {row['nm']}."
    if row['tc'] == 1:
        sent += " Cette école est privée "
    else :
        sent += " Cette école est publique "
    if row['app'] == 1:
        sent += " et propose des formations en apprentissage."
    else:
        sent += "."
    if row['int_f'] and row['int_m']:
        sent += " Cette école propose un internat mixte."
    elif row['int_f']:
        sent += " Cette école propose un internat pour filles."
    elif row['int_m']:
        sent += " Cette école propose un internat pour garçons."
    if row['amg_p'] == 1:
        sent += " Cette école propose des formations en présentiel."
    if row['amg_ed'] == 1:
        sent += " Cette école propose des formations entièrement à distance."
    if row['amg_pd'] == 1:
        sent += " Cette école propose des formations partiellement à distance."
    if row['amg_s'] == 1:
        sent += " Cette école propose des formations pour sportifs."
    if row['amg_a'] == 1:
        sent += " Cette école propose des formations pour artistes."
    return sent

In [65]:
def create_dataframe_description(df):
    df_desc = pd.DataFrame(columns=['description', 'url'])
    df_desc['description'] = df.apply(lambda row: convert_to_description(row), axis=1)
    df_desc['url'] = df['fiche']
    return df_desc

In [66]:
df_desc = create_dataframe_description(formations_dataset)
display(df_desc)
#export
df_desc.to_csv('formations_description.csv', index=False)

,description,url
0,Institut Catholique de Lyon (Lyon 2e Arrondis...,https://dossier.parcoursup.fr/Candidats/public...
1,Institut Catholique de Lyon (Lyon 2e Arrondis...,https://dossier.parcoursup.fr/Candidats/public...
2,Institut Catholique de Lyon (Lyon 2e Arrondis...,https://dossier.parcoursup.fr/Candidats/public...
3,Université de Rennes (EPE) - Site de Saint-Bri...,https://dossier.parcoursup.fr/Candidats/public...
4,"Nantes Université (44), située en Loire-Atlant...",https://dossier.parcoursup.fr/Candidats/public...
...,...,...
105878,"Université Paris Nanterre (92), située en Haut...",NaN
105879,Ecole Universitaire de Premier Cycle - Univers...,NaN
105880,"Lycée Maurice Genevoix (Montrouge - 92), situé...",NaN
105881,Ecole Universitaire de premier cycle - Campus ...,NaN


### Conversion to embeddings

In [67]:
from sentence_transformers import SentenceTransformer

In [68]:
formations_dataset = pd.read_csv('formations_description.csv')
formations_dataset.drop_duplicates(subset=['description'], inplace=True)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(formations_dataset['description'].values)

# export embeddings
pd.DataFrame(embeddings).to_csv('formations_embeddings.csv', index=False)

/home/zatomos/miniconda3/envs/pt/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
